<h3>Capstone Project - The Battle of Neighborhoods</h3>

## Introduction
### Business Problem
<p>My capstone project was very challenging because one need to be original in thought and imagine what could be of interest and useful to stakeholders. Be as it may, i want to provide the geographical coordinates and the map of all the medical facilities within Toronto. this will be of interest to visitor or tourist as well as people who are not very familiar with Toronto Canada. health care is very important and location of healthcare facility is very crucial. In the advent of health emergency, Hospitals and facilities closest to individuals can easily be located and get treated or medical advice obtained. This can  prevent death and and other complications to save life especially With the out break of covid. </p>


## Data 
### Description

Based on the Business problem :

1. Our data sources will be needed to extract/generate the required information:, https://en.wikipedia.org/wiki/List_of_hospitals_in_Toronto
2. Data scrapping will be done using Beautifulsoup python api. 
3. We will transform the data into Pandas dataframe with names of hospitals, District located and othe informations. 
3.  Then we will geocode the Districts to obtain the latitiudes using Foursquare API for further analysis.

### Data Scraping and Data Cleaning

In [70]:
# import the library we use to open URLs
import pandas as pd #import pandas
import numpy as np #Import Numpy
import urllib.request # import .request
!pip install BeautifulSoup4 # install beautifulsoup API
from bs4 import BeautifulSoup # import .Beautifulsoup python API
!pip install geopandas
!pip install geopy
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [71]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_hospitals_in_Toronto"
#open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

In [72]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
#!pip3 install lxml
soup = BeautifulSoup(page, 'html5lib' )

In [73]:
#print(soup.prettify())

In [74]:
soup.title.string

'List of hospitals in Toronto - Wikipedia'

In [75]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable

all_tables=soup.find_all("table")

#all_tables

In [76]:
right_table=soup.find('table', class_='wikitable sortable')

#right_table

In [77]:
A=[]

B=[]

C=[]
D=[]
E=[]
F=[]
G=[]


for row in right_table.findAll('tr'):

    cells=row.findAll('td')

    if len(cells)==7:

        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        D.append(cells[3].find(text=True))
        E.append(cells[4].find(text=True))
        F.append(cells[5].find(text=True))
        G.append(cells[6].find(text=True))


In [78]:


df=pd.DataFrame(A,columns=['Name'])

df['Founded']=B

df['District']=C
df['Network']=D
df['University affiliate']=E
df['Former name(s)']=F
df['Image']=G
df

,Name,Founded,District,Network,University affiliate,Former name(s),Image
0,Baycrest Health Sciences,1918,North York,None,University of Toronto,Toronto Jewish Old Folks Home,\n
1,Bellwood Health Services,1984,East York,Edgewood Health Network,None,None,\n
2,Bridgepoint Active Healthcare,1875,Old Toronto,Sinai Health System,University of Toronto,Riverdale Hospital,\n
3,Casey House,1988,Old Toronto,None,None,None,\n
4,Centre for Addiction and Mental Health,1966,Old Toronto,Centre for Addiction and Mental Health,University of Toronto,Clarke Institute of Psychiatry,\n
5,Centre for Addiction and Mental Health,1850,Old Toronto,Centre for Addiction and Mental Health,University of Toronto,Provincial Lunatic Asylum;,\n
6,Centre for Addiction and Mental Health,1949,Old Toronto,Centre for Addiction and Mental Health,University of Toronto,Addiction Research Foundation,\n
7,Centric Health Surgical Centre Toronto,1960,North York,None,None,Don Mills Surgical Unit,\n
8,Etobicoke General Hospital,1972,Etobicoke,William Osler Health System,None,None,\n
9,Holland Bloorview Kids Rehabilitation Hospital,1899,East York,None,University of Toronto,Home for Incurable Children;,\n


In [79]:
# we slice out Name of hospital, District and network
df_hospital= df[['Name', 'District']]
df_hospital.head()

,Name,District
0,Baycrest Health Sciences,North York
1,Bellwood Health Services,East York
2,Bridgepoint Active Healthcare,Old Toronto
3,Casey House,Old Toronto
4,Centre for Addiction and Mental Health,Old Toronto


In [80]:
df_hospital.shape

(35, 2)

In [81]:
df_hospital['Latitude'] = None
df_hospital['Longitude'] = None
locator = Nominatim(user_agent="myGeocoder")
for i in range(len(df_hospital)):
    location = locator.geocode(df_hospital.loc[i,'District'])
    df_hospital.loc[i,'Latitude'] = location.latitude   
    df_hospital.loc[i,'Longitude'] = location.longitude
df_hospital

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

,Name,District,Latitude,Longitude
0,Baycrest Health Sciences,North York,43.7543,-79.4491
1,Bellwood Health Services,East York,43.7,-79.3325
2,Bridgepoint Active Healthcare,Old Toronto,43.6535,-79.3839
3,Casey House,Old Toronto,43.6535,-79.3839
4,Centre for Addiction and Mental Health,Old Toronto,43.6535,-79.3839
5,Centre for Addiction and Mental Health,Old Toronto,43.6535,-79.3839
6,Centre for Addiction and Mental Health,Old Toronto,43.6535,-79.3839
7,Centric Health Surgical Centre Toronto,North York,43.7543,-79.4491
8,Etobicoke General Hospital,Etobicoke,43.6436,-79.5656
9,Holland Bloorview Kids Rehabilitation Hospital,East York,43.7,-79.3325


### Methodology

#### Data exploratory and analysis 

Our Dataset after scraping, and preprocessing has main features as Hospital name, District, Latitude and Longitude informations of the city of toronto.

In [82]:
#remove duplicate names of hospital
# dropping ALL duplicte values 
df_hospital.drop_duplicates(subset ="Name", 
                     keep = False, inplace = True) 
df_hospital

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Name,District,Latitude,Longitude
0,Baycrest Health Sciences,North York,43.7543,-79.4491
1,Bellwood Health Services,East York,43.7,-79.3325
2,Bridgepoint Active Healthcare,Old Toronto,43.6535,-79.3839
3,Casey House,Old Toronto,43.6535,-79.3839
7,Centric Health Surgical Centre Toronto,North York,43.7543,-79.4491
8,Etobicoke General Hospital,Etobicoke,43.6436,-79.5656
9,Holland Bloorview Kids Rehabilitation Hospital,East York,43.7,-79.3325
10,Hospital for Sick Children,Old Toronto,43.6535,-79.3839
11,Humber River Hospital,North York,43.7543,-79.4491
12,Michael Garron Hospital,East York,43.7,-79.3325


#### Create a map of Toronto Canada with Hospitals superimposed on top.

#### We Used geopy library to get the latitude and longitude values of Toronto, Canada.

#### In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.


In [86]:
address = 'Toronto, canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [87]:
# create map of Toronto Map using latitude and longitude values
map_hospital = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, district, name in zip(df_hospital['Latitude'], df_hospital['Longitude'], df_hospital['District'], df_hospital['Name']):
    label = '{}, {}'.format(name, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hospital)  
    
map_hospital

From the map, we discover only four(4) of the 29 hospitals was captured on the map.This means that all this Hospitals are located within four Districts in toronto canada.

In [88]:
df_hospital_counts = df_hospital['District'].value_counts().to_frame()
df_hospital_counts.rename(columns={'District': 'Number_of_hospitals'}, inplace=True)
df_hospital_counts.index.name = 'District'
df_hospital_counts

,Number_of_hospitals
District,
Old Toronto,13
North York,6
Scarborough,4
East York,3
Etobicoke,2
York,1


In [89]:
df_hospital_counts.dtypes

Number_of_hospitals    int64
dtype: object

In [90]:
df_hospital_district = df_hospital['District'].unique()
df_hospital_district

array(['North York', 'East York', 'Old Toronto', 'Etobicoke',
       'Scarborough', 'York'], dtype=object)

 let's simplify the above map and segment and cluster only the Hospitals in North York. So let's slice the original dataframe and create a new dataframe of the North York data.

In [91]:
North_York_data = df_hospital[df_hospital['District'] == 'North York'].reset_index(drop=True)
North_York_data.head()

,Name,District,Latitude,Longitude
0,Baycrest Health Sciences,North York,43.7543,-79.4491
1,Centric Health Surgical Centre Toronto,North York,43.7543,-79.4491
2,Humber River Hospital,North York,43.7543,-79.4491
3,Sunnybrook Health Sciences Centre,North York,43.7543,-79.4491
4,Sunnybrook Holland Centre,North York,43.7543,-79.4491


In [92]:
East_York_data = df_hospital[df_hospital['District'] == 'East York'].reset_index(drop=True)
East_York_data.head()

,Name,District,Latitude,Longitude
0,Bellwood Health Services,East York,43.7,-79.3325
1,Holland Bloorview Kids Rehabilitation Hospital,East York,43.7,-79.3325
2,Michael Garron Hospital,East York,43.7,-79.3325


In [93]:
Old_Toronto_data = df_hospital[df_hospital['District'] == 'Old Toronto'].reset_index(drop=True)
Old_Toronto_data.head()

,Name,District,Latitude,Longitude
0,Bridgepoint Active Healthcare,Old Toronto,43.6535,-79.3839
1,Casey House,Old Toronto,43.6535,-79.3839
2,Hospital for Sick Children,Old Toronto,43.6535,-79.3839
3,Mount Sinai Hospital,Old Toronto,43.6535,-79.3839
4,Princess Margaret Cancer Centre,Old Toronto,43.6535,-79.3839


### Let Explore Hospitals in each District using Foursquare API

#### we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [104]:
CLIENT_ID = 'WJKTRMEPEYLN3LJX0QR1KVNQSO35UUDMLU31ZB0JB2XBG3FW' # your Foursquare ID
CLIENT_SECRET = 'UD33PRU5LMUHE1HBAUBBS15HYI5J3ES4554ZLQELSB4DX00D' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WJKTRMEPEYLN3LJX0QR1KVNQSO35UUDMLU31ZB0JB2XBG3FW
CLIENT_SECRET:UD33PRU5LMUHE1HBAUBBS15HYI5J3ES4554ZLQELSB4DX00D


Let's explore the first North York dstrict in our dataframe.

Get the various hospitals in North York dstrict.

In [105]:
# The first Hospital within the North York District of toronto City
North_York_data.loc[0, 'Name']

'Baycrest Health Sciences'

In [106]:
# The Third Hospital within the North York District of toronto City
North_York_data.loc[2, 'Name']

'Humber River Hospital'

Get the 's latitude and longitude values of a particular Hospital.

In [107]:
hospital_latitude = North_York_data.loc[0, 'Latitude'] # neighborhood latitude value
hospital_longitude = North_York_data.loc[0, 'Longitude'] # neighborhood longitude value

hospital_name = North_York_data.loc[0, 'Name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(hospital_name, 
                                                               hospital_latitude, 
                                                               hospital_longitude))

Latitude and longitude values of Baycrest Health Sciences are 43.7543263, -79.44911696639593.


Now, we get the top 100 venues that are located within a radius of 500 meters to North York district which could be of interest.

In [110]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    hospital_latitude, 
    hospital_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=WJKTRMEPEYLN3LJX0QR1KVNQSO35UUDMLU31ZB0JB2XBG3FW&client_secret=UD33PRU5LMUHE1HBAUBBS15HYI5J3ES4554ZLQELSB4DX00D&v=20180605&ll=43.7543263,-79.44911696639593&radius=500&limit=100'

In [112]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6005e3de879fa652d7e69a53'},
 'response': {'headerLocation': 'Bathurst Manor',
  'headerFullLocation': 'Bathurst Manor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.758826304500005,
    'lng': -79.44289859530187},
   'sw': {'lat': 43.7498262955, 'lng': -79.45533533749}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5a888f7647f8767d37b92f00',
       'name': 'Grill Gate',
       'location': {'address': '832 Sheppard West',
        'crossStreet': 'NE corner of Sheppard & Wilmington',
        'lat': 43.753122978404804,
        'lng': -79.45169001817703,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.753122978404804,
          'lng': -79.45169001

In [113]:

# create map of Toronto Map using latitude and longitude values
map_north_york = folium.Map(location=[hospital_latitude, hospital_longitude], zoom_start=10)

# add markers to map
for lat, lng, district, name in zip(df_hospital['Latitude'], df_hospital['Longitude'], df_hospital['District'], df_hospital['Name']):
    label = '{}, {}'.format(name, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_north_york)  
    
map_north_york

### Results

From the map, we discover only four(6) of the 29 hospitals was captured on the map. Hospital distribution in each district as follows:

In [94]:
df_hospital_counts = df_hospital['District'].value_counts().to_frame()
df_hospital_counts.rename(columns={'District': 'Number_of_hospitals'}, inplace=True)
df_hospital_counts.index.name = 'District'
df_hospital_counts

,Number_of_hospitals
District,
Old Toronto,13
North York,6
Scarborough,4
East York,3
Etobicoke,2
York,1


### Discussion

We discover that using geocoding API we could append the coordinate to the various District showing the Hospital facilities. furthemore with the foursquare API we can locate this Hospitals using our credentials.

### Conclusion

In this project, we extracted a dataframe with hospital names and districts from wikipedia webage. We use Beautifulsoup API to extact the details. We then Geocode the district to extact the various latitudes and logitudes and append it to the main dataframe.
We Create a map of Toronto Canada with Hospitals superimposed on top. using the Folium API by utilizing the latitudes and the logitudes. the result sho that the 29 distict Hospitals lies on five district or borough. Visitors or tourist can easily locate this hospitals to get medical services in event of an emargency. it can also be useful for anyone not familiar with the tarian.